<a href="https://colab.research.google.com/github/moung1012/Numpy-Pandas/blob/master/Keras3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 영화 리뷰 분류 이진 분류 예제
- 인터넷 영화 데이터베이스로부터 가져온 양극단의 리뷰 5만개로 이루어진 데이터셋
- 훈련 데이터 2만 5000개와 테스트 데이터 2만 5000개로 나뉘어 있고 50%는 부정 50%는 긍정 리뷰로 구성되어 있다
- 같은 데이터에서 훈련하고 테스트 해서는 절대 안된다
- 중요한 것은 새로운 데이터에 대한 모델의 성능

In [ ]:
from keras.datasets import imdb
(train_data, train_labels), (test_data,test_labels) = imdb.load_data(num_words=10000)
# nom_words = 10000 매개 변수는 훈련 데이터에서 가장 자주 나타나는 단어 1만 개만 사용하겠다는 의미
# train_data와 test_data는 리뷰의 목록
# 각 리뷰의 단어 인덱스의 리스트(단어 시퀀스가 인코딩 된것)
#train_labels와 test_labels는 부정을 나타내는 0과 긍정을 나타내는 1의 리스트

In [ ]:
train_data[0]

In [ ]:
train_labels[0]

In [ ]:
max([max(sequence)] for sequence in train_data) # 단어 1만개로 제한했기 때문에 인덱스는 9,999

In [ ]:
word_index = imdb.get_word_index() # word_index는 단어와 정수 인덱스를 매핑한 딕셔너리
revers_word_index= dict(
    [(value, key) for (key, value) in word_index.items()]) # 정수 인덱스와 단어를 매핑하도록 뒤집는다
decoded_review = ' '.join(
    [reverse_word_index.get(i - 3,'?')for i in train_data[0]]) # 리뷰를 디코딩한다 0,1,2는 '패딩','문서 시작','사전에 없음'을 위한 인덱스이므로 3을 뺀다

### 데이터 준비  
리스트를 텐서로 바꾸는 두가지 방법  
- 같은 길이가 되도록 리스트에 패딩을 추가하고 (samples, sequence_length)크기의 정수 텐서로 변환 그 다음 정수 텐서를 다룰 수 있는 층을 신경망의 첫 번째 층으로 사용  
- 리시트를 원-핫 인코딩(one-hot incoding)하여 0과 1의 벡터로 변환한다

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results

# 훈련 데이터를 벡터로 변환합니다
x_train = vectorize_sequences(train_data)
# 테스트 데이터를 벡터로 변환합니다
x_test = vectorize_sequences(test_data)

In [ ]:
x_train[0]

In [ ]:
# 레이블을 벡터로 바꿉니다
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

### 신경망 모델 만들기
- 입력 데이터가 벡터고 레이블은 스칼라(1또는 0)